# Machine Learning terms and metrics

Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [24]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [25]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [26]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [27]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [28]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    print(classes)
    return predlabel


We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [29]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [30]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [31]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [32]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [33]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
[0 1 2 3 4 5]
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [34]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
[0 1 2 3 4 5]
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [35]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [36]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Answer 1

Increasing the validation set size usually reduces accuracy because the model is trained on less data, making it weaker.Decreasing validation set size may show higher accuracy, but it could be less reliable due to fewer and possibly biased samples.



In [37]:
for pct in [80, 75, 70]:  # 20%, 25%, 30% validation
    traindata, trainlabel, valdata, vallabel = split(
        alltraindata, alltrainlabel, pct / 100
    )
    valpred = NN(traindata, trainlabel, valdata)
    valAccuracy = Accuracy(vallabel, valpred)
    print(f"Validation accuracy ({100 - pct}% validation set): {valAccuracy * 100} %")


Validation accuracy (20% validation set): 34.17070217917676 %
Validation accuracy (25% validation set): 33.40735257833703 %
Validation accuracy (30% validation set): 33.66058906030855 %


## Answer 2

A larger training set helps the model learn better, improving generalization. A larger validation set gives a more reliable estimate of test performance, but only if training size remains sufficient. Best results come from a balanced split or cross-validation.

## Answer 3

Although the accuracy is high for 5% validation set, the overall accuracy of 36% is not reliable. Hence, we need a better model.

In practice validation set is usually 20% of the training data set to avoid overfitting.

In [38]:
for pct in [95, 90, 85,80, 75, 70]:
    traindata, trainlabel, valdata, vallabel = split(
        alltraindata, alltrainlabel, pct / 100
    )
    valpred = NN(traindata, trainlabel, valdata)
    valAccuracy = Accuracy(vallabel, valpred)
    print(f"Validation accuracy ({100 - pct}% validation set): {valAccuracy * 100} %")

Validation accuracy (5% validation set): 33.96704689480355 %
Validation accuracy (10% validation set): 32.59668508287293 %
Validation accuracy (15% validation set): 32.8890631125049 %
Validation accuracy (20% validation set): 34.130105900151285 %
Validation accuracy (25% validation set): 33.49174587293647 %
Validation accuracy (30% validation set): 33.145841703495385 %


> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

### Without Scikit Learn

In [39]:
from collections import Counter

def NN3(traindata, trainlabel, query):
    diff = traindata - query
    sq = diff * diff
    dist = sq.sum(1)

    # Get indices of 3 nearest neighbors
    nearest_indices = np.argsort(dist)[:3]
    nearest_labels = trainlabel[nearest_indices]

    # Return the most common label
    label_counts = Counter(nearest_labels)
    return label_counts.most_common(1)[0][0]

def NN3_pred(traindata, trainlabel, testdata):
    predlabel = np.array([NN3(traindata, trainlabel, i) for i in testdata])
    return predlabel

## Validation dataset of 80%
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 80 / 100)
valpred = NN3_pred(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

## Testing
testpred = NN3_pred(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 35.493732803424024 %
Test accuracy: 36.486486486486484 %


## With Scikit learn

In [40]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 1-NN model
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(traindata, trainlabel)
pred1 = knn1.predict(testdata)
acc1 = accuracy_score(testlabel, pred1)

# 3-NN model
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(traindata, trainlabel)
pred3 = knn3.predict(testdata)
acc3 = accuracy_score(testlabel, pred3)

# Print results
print(f"1-NN Test Accuracy: {acc1 * 100}%")
print(f"3-NN Test Accuracy: {acc3 * 100}%")


1-NN Test Accuracy: 35.32818532818533%
3-NN Test Accuracy: 36.076254826254825%


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [41]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [42]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.824843372875726 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


## Answers

In [43]:
split_percent = 0.8
runs = 5

def run_avg_acc(data, labels, splitpercent, iterations, runs=5, classifier=NN):
    avg_accuracies = []
    for _ in range(runs):
        avg_acc = AverageAccuracy(data, labels, splitpercent, iterations, classifier)
        avg_accuracies.append(avg_acc)
    return np.array(avg_accuracies)

for iterations in [1, 5, 10]:
    results = run_avg_acc(alltraindata, alltrainlabel, split_percent, iterations, runs=5)
    print(f"Iterations: {iterations}")
    print(f"Mean accuracy: {results.mean() * 100}%")
    print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")
    print("------------------")

Iterations: 1
Mean accuracy: 34.61078510412906%
Test accuracy: 34.91795366795367 %
------------------
Iterations: 5
Mean accuracy: 33.9310342596772%
Test accuracy: 34.91795366795367 %
------------------
Iterations: 10
Mean accuracy: 34.212859064730935%
Test accuracy: 34.91795366795367 %
------------------


1. Averaging across multiple splits reduces the effect of randomness in a single split, leading to more stable and consistent validation accuracy.

2. Yes, to an extent. The averaged validation accuracy tends to be closer to the true test accuracy compared to relying on just one split. But it's still just an estimate.

3.  Yes. Increasing iterations generally gives a better (more reliable and stable) estimate of validation accuracy, as the influence of outlier splits is reduced.

4. Partially. More iterations help average out variability from small datasets, but they cannot replace the need for enough data. Too little training data can still lead to poor generalization.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

In [44]:
def knn_classifier(k):
    def classifier(train_X, train_y, test_X):
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(train_X, train_y)
        return knn.predict(test_X)
    return classifier

def test_knn_vary_split_size(data, labels, split_sizes, iterations=10):
    print("Testing varying split sizes")
    for split_percent in split_sizes:
        avg_acc_1nn = AverageAccuracy(data, labels, split_percent, iterations, classifier=knn1)
        avg_acc_3nn = AverageAccuracy(data, labels, split_percent, iterations, classifier=knn3)
        print(f"Train split: {split_percent*100}% | 1-NN Acc: {avg_acc_1nn*100}% | 3-NN Acc: {avg_acc_3nn*100}%")

def test_knn_vary_iterations(data, labels, split_percent=0.75, max_iterations=20):
    print("Testing varying number of splits (iterations)")
    for iter_count in range(1, max_iterations + 1):
        avg_acc_1nn = AverageAccuracy(data, labels, split_percent, iter_count, classifier=knn1)
        avg_acc_3nn = AverageAccuracy(data, labels, split_percent, iter_count, classifier=knn3)
        print(f"Iterations: {iter_count} | 1-NN Acc: {avg_acc_1nn*100}% | 3-NN Acc: {avg_acc_3nn*100}%")

knn1 = knn_classifier(1)
knn3 = knn_classifier(3)

split_sizes = [0.5, 0.6, 0.7, 0.8, 0.9]  # different train split percentages

test_knn_vary_split_size(alltraindata, alltrainlabel, split_sizes, iterations=10)

test_knn_vary_iterations(alltraindata, alltrainlabel, split_percent=0.75, max_iterations=15)

Testing varying split sizes
Train split: 50.0% | 1-NN Acc: 33.11520879896378% | 3-NN Acc: 33.5231473117317%
Train split: 60.0% | 1-NN Acc: 33.46270761428339% | 3-NN Acc: 33.732593157113996%
Train split: 70.0% | 1-NN Acc: 34.05485469320069% | 3-NN Acc: 34.13569703855108%
Train split: 80.0% | 1-NN Acc: 34.05879972200539% | 3-NN Acc: 34.512866832557194%
Train split: 90.0% | 1-NN Acc: 34.821679970092156% | 3-NN Acc: 35.08896611431536%
Testing varying number of splits (iterations)
Iterations: 1 | 1-NN Acc: 33.3252427184466% | 3-NN Acc: 34.4770468528159%
Iterations: 2 | 1-NN Acc: 33.715454547617234% | 3-NN Acc: 34.22433884524892%
Iterations: 3 | 1-NN Acc: 33.80109285223804% | 3-NN Acc: 34.400429162976145%
Iterations: 4 | 1-NN Acc: 33.77580056576609% | 3-NN Acc: 34.57652985338263%
Iterations: 5 | 1-NN Acc: 34.006659993209% | 3-NN Acc: 34.52918518214238%
Iterations: 6 | 1-NN Acc: 34.234316474686665% | 3-NN Acc: 34.649794004066464%
Iterations: 7 | 1-NN Acc: 34.232377801021755% | 3-NN Acc: 34.51